In [1]:
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk

from database import get_high_scores, get_top_scores, insert_new

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - \
        np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle

    return angle


In [3]:
def exerciser(options,choice):
    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0
    stage = None
    
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                counter,stage=options[choice](counter,stage,landmarks,image)

            except:
                pass

            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

            # Rep data
            cv2.putText(image, 'REPS', (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter),
                        (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

            # Stage data
            cv2.putText(image, 'STAGE', (65, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, stage,
                        (60, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    return counter

In [4]:
def squat(counter,stage,landmarks,image):
        # Get coordinates
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]

        # Calculate angle
        angle_knee= calculate_angle(hip, knee, ankle)
        angle_ankle = calculate_angle(knee, ankle, heel)

        # Visualize angle
        cv2.putText(image, str(angle_knee),
                tuple(np.multiply(knee, [640, 480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,
                                                255, 255), 2, cv2.LINE_AA
                )
        cv2.putText(image, str(angle_ankle),
                tuple(np.multiply(ankle, [640, 480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,
                                                255, 255), 2, cv2.LINE_AA
                )
        # Squats raise counter logic
        if angle_knee > 170:
                stage = "up"
        if angle_knee < 90 and stage == 'up':
                stage = "down"
                counter += 1
                print(counter)
        return counter,stage

In [5]:
def lateral(counter,stage,landmarks,image):
        # Get coordinates
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

        # Calculate angle
        angle_elbow = calculate_angle(shoulder, elbow, wrist)
        angle_shoulder = calculate_angle(hip, shoulder, elbow)

        # Visualize angle
        cv2.putText(image, str(angle_elbow),
                tuple(np.multiply(elbow, [640, 480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255, 255), 2,
                cv2.LINE_AA)
        cv2.putText(image, str(angle_shoulder),
                tuple(np.multiply(shoulder, [640, 480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255, 255), 2,
                cv2.LINE_AA)
        # Lateral raise counter logic
        if angle_elbow > 160 and angle_elbow < 180:
                if angle_shoulder > 90:
                        stage = "down"
                if angle_shoulder < 30 and stage == 'down':
                        stage = "up"
                        counter += 1
                        print(counter)
        return counter,stage

In [6]:
def curl(counter,stage,landmarks,image):
        # Get coordinates
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        # Calculate angle
        angle = calculate_angle(shoulder, elbow, wrist)

        # Visualize angle
        cv2.putText(image, str(angle),
                tuple(np.multiply(elbow, [640, 480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255, 255), 2, cv2.LINE_AA
                )

        # Curl counter logic
        if angle > 160:
                stage = "down"
        if angle < 30 and stage == 'down':
                stage = "up"
                counter += 1
                print(counter)
        return counter,stage

In [7]:
# Create the main window
window = tk.Tk()

# Get the screen width and height
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()

# Set the window size to full screen--------------------------------------------------
window.geometry(f"{screen_width}x{screen_height}")
width1 = (screen_width*2)/3
width2 = screen_width/3
height= screen_height-100
# Create the heading rectangle frame
heading_frame = tk.Frame(window, bg="gray", width=screen_width, height=100)
heading_frame.place(x=0, y=0)

# Add text to the heading frame
heading_label = tk.Label(heading_frame, text="Welcome Dikshit", font=(
    "Arial", 24), fg="white", bg="gray")
heading_label.place(relx=0.5, rely=0.5, anchor="center")
#-----------------------------------FRAMES----------------------------------------------------
# Create the first section frame
frame1 = tk.Frame(window, bg="red", width=width1, height=height)
frame1.place(x=0, y=100)

# Create the second section frame
frame2 = tk.Frame(window, bg="blue", width=width2, height=height/3)
frame2.place(x=width1, y=100)

# Create the third section frame
frame3 = tk.Frame(window, bg="green", width=width2, height=height/3)
frame3.place(x=width1, y=100 + height/3)


# Create the fourth section frame
frame4 = tk.Frame(window, bg="yellow", width=width2, height=height/3)
frame4.place(x=width1, y=100 + (height*2)/3)

#------------------------------FRAME1---------------------------------------------
# Open a video capture
cap = cv2.VideoCapture(0)

# Function to update video frames

# Create an instance of mediapipe pose
pose = mp_pose.Pose(min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

def update_video():

    global frame1, cap, counter, stage

    ret, frame = cap.read()
    frame = cv2.resize(frame, (frame1.winfo_width(), frame1.winfo_height()))

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make detection
    results = pose.process(image)

    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark

        # Get coordinates
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        # Calculate angle
        angle = calculate_angle(shoulder, elbow, wrist)

        # Visualize angle
        cv2.putText(image, str(angle),
                    tuple(np.multiply(elbow, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,
                                                    255, 255), 2, cv2.LINE_AA
                    )

        # Curl counter logic
        if angle > 160:
            stage = "down"
        if angle < 30 and stage == 'down':
            stage = "up"
            counter += 1
            print(counter)

    except:
        pass

    # Render curl counter
    # Setup status box
    cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

    # Rep data
    cv2.putText(image, 'REPS', (15, 12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(counter),
                (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

    # Stage data
    cv2.putText(image, 'STAGE', (65, 12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, stage,
                (60, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(
                                color=(245, 117, 66), thickness=2, circle_radius=2),
                            mp_drawing.DrawingSpec(
                                color=(245, 66, 230), thickness=2, circle_radius=2)
                            )
    # Convert the image to PIL format
    img = Image.fromarray(image)

    # Create PhotoImage object to display in the label
    img_tk = ImageTk.PhotoImage(img)

    # Update the label with the new image
    label.config(image=img_tk)
    label.image = img_tk

    # Schedule the next update
    if window.winfo_exists():
        frame1.after(10, update_video)
    else:
        cap.release()

# Add label to display video frames
label = tk.Label(frame1)
label.place(x=0, y=0, relwidth=1, relheight=1)

#----------------------------------------------FRAME2--------------------------------------------------
# Sample data
data = get_high_scores()

# Create a treeview for displaying the leaderboard
leaderboard_treeview = ttk.Treeview(frame2, columns=(
    "Exercise", "Player", "High Score"), show="headings")

# Define column headings
leaderboard_treeview.heading("Exercise", text="Exercise")
leaderboard_treeview.heading("Player", text="Player")
leaderboard_treeview.heading("High Score", text="High Score")

# Insert data into the treeview
for item in data:
    exercise, first_name, last_name, high_score = item
    player_name = f"{first_name} {last_name}"
    leaderboard_treeview.insert("", tk.END, values=(
        exercise, player_name, high_score))

# Create a treeview for displaying the leaderboard
leaderboard_treeview = ttk.Treeview(frame2, columns=(
    "Exercise", "Player", "High Score"), show="headings")

# Define column headings
leaderboard_treeview.heading("Exercise", text="Exercise")
leaderboard_treeview.heading("Player", text="Player")
leaderboard_treeview.heading("High Score", text="High Score")

# Set the column widths
leaderboard_treeview.column("Exercise", width=int(width2//3))
leaderboard_treeview.column("Player", width=int(width2//3))
leaderboard_treeview.column("High Score", width=int(width2//3))

# Insert data into the treeview
for item in data:
    exercise, first_name, last_name, high_score = item
    player_name = f"{first_name} {last_name}"
    leaderboard_treeview.insert("", tk.END, values=(
        exercise, player_name, high_score))

# Configure grid layout for leaderboard_treeview
leaderboard_treeview.grid(row=0, column=0, sticky="nsew")

# Configure weight of the rows and columns
frame2.rowconfigure(0, weight=1)
frame2.columnconfigure(0, weight=1)

# Add a vertical scrollbar to the treeview
scrollbar = ttk.Scrollbar(frame2, orient="vertical",
                          command=leaderboard_treeview.yview)
scrollbar.grid(row=0, column=1, sticky="ns")

# Configure scrollbar for the treeview
leaderboard_treeview.configure(yscrollcommand=scrollbar.set)


#------------------------------------------FRAME3-----------------------------------------------------
label3 = tk.Label(frame3, text="Top 5 per exercise", fg="white", bg="green")
label3.pack(fill=tk.X)

# Retrieve the top scores for each exercise category
data = get_top_scores()

# Create a drop-down menu for exercises
exercise_var = tk.StringVar()
exercise_combobox = ttk.Combobox(frame3, textvariable=exercise_var)
exercise_combobox['values'] = sorted(set(item[0] for item in data))
exercise_combobox.pack()

# Create a treeview for displaying scores
score_treeview = ttk.Treeview(
    frame3, columns=("Name", "Score"), show="headings")

# Define column headings
score_treeview.heading("Name", text="Name")
score_treeview.heading("Score", text="Highscore")

# Set the column widths to fit the available width
score_treeview.column("Name", width=int(width2/2))
score_treeview.column("Score", width=int(width2/2))

def update_scores():
    # Clear previous scores
    score_treeview.delete(*score_treeview.get_children())

    # Get the selected exercise
    selected_exercise = exercise_var.get()

    # Get the top 5 scores for the selected exercise
    top_scores = [(item[1] + " " + item[2], item[3])
                  for item in data if item[0] == selected_exercise][:5]

    # Insert the scores into the treeview
    for score in top_scores:
        name, highscore = score
        score_treeview.insert("", tk.END, values=(name, highscore))


# Update scores when exercise is selected
exercise_combobox.bind("<<ComboboxSelected>>", lambda event: update_scores())

# Pack the treeview
score_treeview.pack(fill=tk.BOTH, expand=True)

#------------------------------------------FRAME4-----------------------------------------------------


label4 = tk.Label(frame4, text="Section 4", fg="black", bg="yellow")
label4.pack()
# Initialize counter and stage
counter = 0
stage = None

# Start updating video frames
update_video()

# Bind the camera release to window closing event


def release_camera():
    if cap.isOpened():
        cap.release()
    window.destroy()


window.protocol("WM_DELETE_WINDOW", release_camera)

# Start the Tkinter event loop
window.mainloop()

1
2
3
